## The idea of this workflow in the first place is to take as much structured data from different APIs as possible before matching a building against the GND. Information from OpenStreetMap or via an already present GND identifier are stored in a second dataframe for later use to enrich the original dataset or to compare a rich GND-dataset to the original one to find out, if it is a correct match.

# Matching A

## Step 1: If coordinates are present, search using coordinates, else search using address and/or building name via OSM

In [1]:
import requests
import json
import pandas as pd
from IPython.display import display
from IPython.display import HTML
pd.set_option('display.max_colwidth', None)

#coordinates    
lat = "50.9309068"
lon = "11.5873241"

#alternative: search for address and name, take first item of response only
address = "bibliotheksplatz 2 jena"
name = "ThULB Jena"
if (address != '') and (name != ''):
    address_name_search = 'https://nominatim.openstreetmap.org/search?q=' + address + name + '&format=jsonv2&extratags=1'
elif address != '':
    address_name_search = 'https://nominatim.openstreetmap.org/search?q=' + address + '&format=jsonv2&extratags=1'
elif name != '':
    address_name_search = 'https://nominatim.openstreetmap.org/search?q=' + name + '&format=jsonv2&extratags=1'

url = address_name_search

#search using coordinates
url = 'https://nominatim.openstreetmap.org/reverse.php?lat=' + lat + '&lon=' + lon + '&zoom=18&format=jsonv2&&extratags=1'
response = requests.get(url)
data = json.loads(response.text)
print(data)
data = pd.json_normalize(data)
df = pd.DataFrame(data)

display(df)


{'place_id': 156656309, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 201322986, 'lat': '50.9305666', 'lon': '11.587797099528302', 'place_rank': 30, 'category': 'amenity', 'type': 'library', 'importance': 0.2862449547378125, 'addresstype': 'amenity', 'name': 'Thüringer Universitäts und Landesbibliothek (ThULB)', 'display_name': 'Thüringer Universitäts und Landesbibliothek (ThULB), 2, Bibliotheksplatz, Zentrum, Jena-Zentrum, Jena, Thüringen, 07743, Deutschland', 'address': {'amenity': 'Thüringer Universitäts und Landesbibliothek (ThULB)', 'house_number': '2', 'road': 'Bibliotheksplatz', 'suburb': 'Zentrum', 'city_district': 'Jena-Zentrum', 'city': 'Jena', 'state': 'Thüringen', 'ISO3166-2-lvl4': 'DE-TH', 'postcode': '07743', 'country': 'Deutschland', 'country_code': 'de'}, 'extratags': {'operator': 'Friedrich-Schiller-Universität Jena', 'ref:isil': 'DE-27', 'wikidata': 'Q1255096', 'wikipedia': 'de:Thüringer Universitäts-

,place_id,licence,osm_type,osm_id,lat,lon,place_rank,category,type,importance,...,extratags.wikidata,extratags.wikipedia,extratags.wheelchair,extratags.contact:phone,extratags.opening_hours,extratags.building:levels,extratags.contact:website,extratags.internet_access,extratags.air_conditioning,extratags.toilets:wheelchair
0,156656309,"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",way,201322986,50.9305666,11.587797099528302,30,amenity,library,0.286245,...,Q1255096,de:Thüringer Universitäts- und Landesbibliothek,yes,+49 3641 940 100,Mo-Fr 09:00-22:00; Sa 10:00-18:00,4,http://www.thulb.uni-jena.de/,wlan,yes,yes


## Step 2: get interesting key:value-pairs for enrichment/later comparison

In [2]:
enrichment = df[['place_id','lat','lon', 'name', 'address.road','address.house_number', 'address.city_district', 'address.city', 'address.state', 'address.ISO3166-2-lvl4', 'extratags.wikidata']]

display(enrichment)

,place_id,lat,lon,name,address.road,address.house_number,address.city_district,address.city,address.state,address.ISO3166-2-lvl4,extratags.wikidata
0,156754009,50.9305666,11.587797099528302,Thüringer Universitäts und Landesbibliothek (ThULB),Bibliotheksplatz,2,Jena-Zentrum,Jena,Thüringen,DE-TH,Q1255096


## Step 3: If there is a wikidata-ID, get GND-ID via wikidata, else go to step 4
via extratags.wikidata

In [3]:
wikidata_id = "Q1255096"

url = 'https://www.wikidata.org/w/api.php?action=wbgetentities&ids=' + wikidata_id + '&format=json&language=en&type=item'
response = requests.get(url)
data = json.loads(response.text)

gndIdentifier = data['entities'][wikidata_id]['claims']['P227'][0]['mainsnak']['datavalue']['value']

display(gndIdentifier)

'5053491-9'

In [4]:
enrichment['gndIdentifier'] = gndIdentifier
display(enrichment)

/var/folders/72/m72jw2752rd7brgxncwc56cw0000gn/T/ipykernel_13869/3237667349.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enrichment['gndIdentifier'] = gndIdentifier


,place_id,lat,lon,name,address.road,address.house_number,address.city_district,address.city,address.state,address.ISO3166-2-lvl4,extratags.wikidata,gndIdentifier
0,156754009,50.9305666,11.587797099528302,Thüringer Universitäts und Landesbibliothek (ThULB),Bibliotheksplatz,2,Jena-Zentrum,Jena,Thüringen,DE-TH,Q1255096,5053491-9


## Step 4: If there is a GND-ID from wikidata, get data from GND via lobid. If there already was a GND-Identifier in the source data, compare both and search for both, if different. Save the response(s) in the enrichment dataframe. Else go to step 8

In [5]:
url = 'http://lobid.org/gnd/' + gndIdentifier + '.json'
response = requests.get(url)

gnd_data = json.loads(response.text)
gnd_data = pd.json_normalize(gnd_data)
gnd_data = gnd_data[['preferredName', 'geographicAreaCode', 'placeOfBusiness', 'broaderTermInstantial', 'gndSubjectCategory']]

#gnd_data['geographicAreaCode'] = gnd_data['geographicAreaCode'].apply(lambda x: [x[0]['id'] + ' - ' + x[0]['label']])[0]
#gnd_data['placeOfBusiness'] = gnd_data['placeOfBusiness'].apply(lambda x: [x[0]['id']+': '+x[0]['label']])[0]
#gnd_data['geographicAreaCode'] = gnd_data['geographicAreaCode'].str.replace("https://d-nb.info/standards/vocab/gnd/geographic-area-code#", "")
#gnd_data['broaderTermInstantial'] = gnd_data['broaderTermInstantial'].apply(lambda x: [i['id']+': '+i['label'] for i in x])
#gnd_data['gndSubjectCategory'] = gnd_data['gndSubjectCategory'].apply(lambda x: [i['id']+': '+i['label'] for i in x])
display(gnd_data)

,preferredName,geographicAreaCode,placeOfBusiness,broaderTermInstantial,gndSubjectCategory
0,Thüringer Universitäts- und Landesbibliothek Jena,"[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-TH', 'label': 'Thüringen'}]","[{'id': 'https://d-nb.info/gnd/4028557-1', 'label': 'Jena'}]","[{'id': 'https://d-nb.info/gnd/4166544-2', 'label': 'Landesbibliothek'}, {'id': 'https://d-nb.info/gnd/4133334-2', 'label': 'Universitätsbibliothek'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#6.6', 'label': 'Hochschule'}, {'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#6.7', 'label': 'Bibliothek, Information und Dokumentation'}]"


## Step 5: Add the data from GND to the enrichment dataframe

In [6]:
enrichment = pd.concat([enrichment, gnd_data[['preferredName','geographicAreaCode', 'placeOfBusiness', 'broaderTermInstantial', 'gndSubjectCategory']]], axis=1)

display(enrichment)

,place_id,lat,lon,name,address.road,address.house_number,address.city_district,address.city,address.state,address.ISO3166-2-lvl4,extratags.wikidata,gndIdentifier,preferredName,geographicAreaCode,placeOfBusiness,broaderTermInstantial,gndSubjectCategory
0,156754009,50.9305666,11.587797099528302,Thüringer Universitäts und Landesbibliothek (ThULB),Bibliotheksplatz,2,Jena-Zentrum,Jena,Thüringen,DE-TH,Q1255096,5053491-9,Thüringer Universitäts- und Landesbibliothek Jena,"[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-TH', 'label': 'Thüringen'}]","[{'id': 'https://d-nb.info/gnd/4028557-1', 'label': 'Jena'}]","[{'id': 'https://d-nb.info/gnd/4166544-2', 'label': 'Landesbibliothek'}, {'id': 'https://d-nb.info/gnd/4133334-2', 'label': 'Universitätsbibliothek'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#6.6', 'label': 'Hochschule'}, {'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#6.7', 'label': 'Bibliothek, Information und Dokumentation'}]"


# Scoring A

## Step 6: If a GND entry was found, compare values from the enrichment-table to the original dataset and enrich it with values/columns that are not present there, else give a warning, if the values don't match
- There is no concept for comparison yet, in the case of identifiers and strings it can be done using string-similarity methods, I think. @Erdal: Please make a suggestion
- For coordinate comparison the values should be identical down to the 4th decimal place which equals to roughly 10 meters, see https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude 
- it would be great, if we could apply javascript-based cell coloring with AgGrid for the comparisons like
    - green for identical
    - orange for similar
    - red for completely different (if both != NaN)

## else if no GND entry was found go to Step 8


## Step 7: User confirms match using the comparison results
Should be similar to person "results" part
"Match", "Possible Match", "No Match"

## if != "Match":

# Matching B

## Step 8: Search via lobid-GND
Use name element + place (city) the building is in. Search via lobid.api for PlaceOrGeographicName but excluding TerritorialCorporateBodyOrAdministrativeUnit

In [14]:
searchterms = "rathaus leipzig"

url = 'https://lobid.org/gnd/search?q=' + searchterms + '&filter=%2B%28type%3APlaceOrGeographicName%29+%2B+NOT+%28type%3ATerritorialCorporateBodyOrAdministrativeUnit%29'
response = requests.get(url)
gnd_data = json.loads(response.text)
gnd_data = pd.json_normalize(gnd_data, 'member')

display(gnd_data)

,type,@context,gndSubjectCategory,oldAuthorityNumber,geographicAreaCode,broaderTermInstantial,gndIdentifier,id,place,preferredName,...,describedBy.license.id,describedBy.license.label,describedBy.dateModified,describedBy.id,deprecatedUri,buildingOwner,hierarchicalSuperiorOfPlaceOrGeographicName,dateOfProduction,biographicalOrHistoricalInformation,variantName
0,"[BuildingOrMemorial, PlaceOrGeographicName, AuthorityResource]",http://lobid.org/gnd/context.jsonld,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]",[(DE-588c)4392110-3],"[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4176999-5', 'label': 'Rathaus'}]",4392110-3,https://d-nb.info/gnd/4392110-3,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Neues Rathaus Leipzig (Leipzig),...,http://creativecommons.org/publicdomain/zero/1.0/,http://creativecommons.org/publicdomain/zero/1.0/,2020-12-07T15:38:44.000,https://d-nb.info/gnd/4392110-3/about,NaN,NaN,NaN,NaN,NaN,NaN
1,"[PlaceOrGeographicName, BuildingOrMemorial, AuthorityResource]",http://lobid.org/gnd/context.jsonld,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[(DE-588c)4369458-5, (DE-588)2154210-7, (DE-588b)2154210-7]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4176999-5', 'label': 'Rathaus'}]",4369458-5,https://d-nb.info/gnd/4369458-5,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Altes Rathaus Leipzig (Leipzig),...,http://creativecommons.org/publicdomain/zero/1.0/,http://creativecommons.org/publicdomain/zero/1.0/,2021-05-03T14:50:45.000,https://d-nb.info/gnd/4369458-5/about,[https://d-nb.info/gnd/2154210-7],NaN,NaN,NaN,NaN,NaN
2,"[BuildingOrMemorial, AuthorityResource, PlaceOrGeographicName]",http://lobid.org/gnd/context.jsonld,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]",NaN,"[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4312641-8', 'label': 'Festsaal'}]",1192325486,https://d-nb.info/gnd/1192325486,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Altes Rathaus Leipzig (Leipzig) / Festsaal,...,http://creativecommons.org/publicdomain/zero/1.0/,http://creativecommons.org/publicdomain/zero/1.0/,2021-05-03T14:51:04.000,https://d-nb.info/gnd/1192325486/about,NaN,"[{'id': 'https://d-nb.info/gnd/118928651', 'label': 'Lotter, Hieronymus'}]","[{'id': 'https://d-nb.info/gnd/4369458-5', 'label': 'Altes Rathaus Leipzig (Leipzig)'}]",NaN,NaN,NaN
3,"[AuthorityResource, BuildingOrMemorial, PlaceOrGeographicName]",http://lobid.org/gnd/context.jsonld,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#3.5a', 'label': 'Liturgik, Frömmigkeit'}, {'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]",[(DE-588c)7755716-5],"[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4073436-5', 'label': 'Kirchenbau'}]",7755716-5,https://d-nb.info/gnd/7755716-5,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Sankt Trinitatis (Leipzig),...,http://creativecommons.org/publicdomain/zero/1.0/,http://creativecommons.org/publicdomain/zero/1.0/,2021-12-17T17:07:07.000,https://d-nb.info/gnd/7755716-5/about,NaN,NaN,NaN,[1847],"[Zweite Trinitatiskirche Leipzig errichtet als Propsteikirche - 1982 fertiggestellt u. geweiht - im Jahr 2015 als Kirche entweiht u. Gebäude unter Denkmalschutz gestellt., Erste Trinitatiskirche (Alte Trinitatiskirche, Römisch-Katholische Propsteikirche zur Heiligen Dreifaltigkeit) errichtet im Jahre 1847 westlich des Neuen Rathauses in der Rudolphstraße - 1943 schwer beschädigt, Reste 1954 gesprengt., Dritte Trinitatiskirche 

## get relevant columns for comparison to original dataset

In [15]:
new_enrichment = gnd_data[['gndSubjectCategory', 'geographicAreaCode', 'broaderTermInstantial', 'id', 'place', 'preferredName', 'variantName', 'dateOfProduction', 'biographicalOrHistoricalInformation']]

display(new_enrichment)

,gndSubjectCategory,geographicAreaCode,broaderTermInstantial,id,place,preferredName,variantName,dateOfProduction,biographicalOrHistoricalInformation
0,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4176999-5', 'label': 'Rathaus'}]",https://d-nb.info/gnd/4392110-3,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Neues Rathaus Leipzig (Leipzig),NaN,NaN,NaN
1,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4176999-5', 'label': 'Rathaus'}]",https://d-nb.info/gnd/4369458-5,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Altes Rathaus Leipzig (Leipzig),NaN,NaN,NaN
2,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4312641-8', 'label': 'Festsaal'}]",https://d-nb.info/gnd/1192325486,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Altes Rathaus Leipzig (Leipzig) / Festsaal,NaN,NaN,NaN
3,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#3.5a', 'label': 'Liturgik, Frömmigkeit'}, {'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4073436-5', 'label': 'Kirchenbau'}]",https://d-nb.info/gnd/7755716-5,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Sankt Trinitatis (Leipzig),"[St. Trinitatis (Leipzig), Katholische Probsteikirche St. Trinitatis (Leipzig), Probsteikirche St. Trinitatis (Leipzig), Alte Trinitatiskirche (Leipzig), Trinitatiskirche Leipzig, Probsteikirche Sankt Trinitatis (Leipzig), Römisch-Katholische Propsteikirche zur Heiligen Dreifaltigkeit (Leipzig), Katholische Probsteikirche Zur Heiligen Dreifaltigkeit (Leipzig)]",[1847],"[Zweite Trinitatiskirche Leipzig errichtet als Propsteikirche - 1982 fertiggestellt u. geweiht - im Jahr 2015 als Kirche entweiht u. Gebäude unter Denkmalschutz gestellt., Erste Trinitatiskirche (Alte Trinitatiskirche, Römisch-Katholische Propsteikirche zur Heiligen Dreifaltigkeit) errichtet im Jahre 1847 westlich des Neuen Rathauses in der Rudolphstraße - 1943 schwer beschädigt, Reste 1954 gesprengt., Dritte Trinitatiskirche Leipzig als Propsteikirche St. Trinitatis errichtet an der Nonnenmühlgasse gegenüber dem Neuen Rathaus - geweiht im Jahr 2015.]"


# Scoring B
see Scoring A

# Before Export

## check place for succeedingPlaceOrGeographicName, use for enrichment, if it is present

In [16]:
#place_id = new_enrichment['place'].apply(lambda x: [x[0]['id']][0]).iloc[0].replace("https://d-nb.info/gnd/", "")

place_id = "11498-4"
url = 'http://lobid.org/gnd/' + place_id + '.json'
response = requests.get(url)
gnd_data = json.loads(response.text)
gnd_data = pd.json_normalize(gnd_data)

if 'succeedingPlaceOrGeographicName' in gnd_data.columns:
    new_enrichment['succeedingPlaceOrGeographicName'] = gnd_data['succeedingPlaceOrGeographicName']

    display(new_enrichment)

,gndSubjectCategory,geographicAreaCode,broaderTermInstantial,id,place,preferredName,variantName,dateOfProduction,biographicalOrHistoricalInformation,succeedingPlaceOrGeographicName
0,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4176999-5', 'label': 'Rathaus'}]",https://d-nb.info/gnd/4392110-3,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Neues Rathaus Leipzig (Leipzig),NaN,NaN,NaN,"[{'id': 'https://d-nb.info/gnd/4094703-8', 'label': 'Staat Hannover'}]"
1,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4176999-5', 'label': 'Rathaus'}]",https://d-nb.info/gnd/4369458-5,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Altes Rathaus Leipzig (Leipzig),NaN,NaN,NaN,NaN
2,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4312641-8', 'label': 'Festsaal'}]",https://d-nb.info/gnd/1192325486,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Altes Rathaus Leipzig (Leipzig) / Festsaal,NaN,NaN,NaN,NaN
3,"[{'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#3.5a', 'label': 'Liturgik, Frömmigkeit'}, {'id': 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3ab', 'label': 'Ortsgebundene Bauwerke'}]","[{'id': 'https://d-nb.info/standards/vocab/gnd/geographic-area-code#XA-DE-SN', 'label': 'Sachsen'}]","[{'id': 'https://d-nb.info/gnd/4073436-5', 'label': 'Kirchenbau'}]",https://d-nb.info/gnd/7755716-5,"[{'id': 'https://d-nb.info/gnd/4035206-7', 'label': 'Leipzig'}]",Sankt Trinitatis (Leipzig),"[St. Trinitatis (Leipzig), Katholische Probsteikirche St. Trinitatis (Leipzig), Probsteikirche St. Trinitatis (Leipzig), Alte Trinitatiskirche (Leipzig), Trinitatiskirche Leipzig, Probsteikirche Sankt Trinitatis (Leipzig), Römisch-Katholische Propsteikirche zur Heiligen Dreifaltigkeit (Leipzig), Katholische Probsteikirche Zur Heiligen Dreifaltigkeit (Leipzig)]",[1847],"[Zweite Trinitatiskirche Leipzig errichtet als Propsteikirche - 1982 fertiggestellt u. geweiht - im Jahr 2015 als Kirche entweiht u. Gebäude unter Denkmalschutz gestellt., Erste Trinitatiskirche (Alte Trinitatiskirche, Römisch-Katholische Propsteikirche zur Heiligen Dreifaltigkeit) errichtet im Jahre 1847 westlich des Neuen Rathauses in der Rudolphstraße - 1943 schwer beschädigt, Reste 1954 gesprengt., Dritte Trinitatiskirche Leipzig als Propsteikirche St. Trinitatis errichtet an der Nonnenmühlgasse gegenüber dem Neuen Rathaus - geweiht im Jahr 2015.]",NaN


## check all broaderTermInstantial if gndSubjectCategory is 31.3a (Architektur)
give feedback if this is not the case

In [3]:
broaderTerm_id = "4176999-5"
#broaderTerm_id = "4030702-5"

url = 'http://lobid.org/gnd/' + broaderTerm_id + '.json'
response = requests.get(url)

if 'https://d-nb.info/standards/vocab/gnd/gnd-sc#31.3a' in response.text:
    print('Es ist alles gut.')
else:
    print('Achtung: Kein Architekturbegriff!')

Es ist alles gut.
